In [31]:
from sklearn.preprocessing import MinMaxScaler 
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import linear_model
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

#### Importing data

In [32]:
df = pd.read_csv('Clean_Data/clean_data.csv')
# df.drop(['Unnamed: 0'], axis=1, inplace=True)
df.head()

,time,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run-of-river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,...,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,weather_main,weather_description,clean_energy_total,fossil_energies_total
0,2015-01-01 00:00:00+01:00,447.0,329.0,4844.0,4821.0,162.0,863.0,1051.0,1899.0,7096.0,...,0.971,63,1,309,0,800,clear,sky is clear,17409.0,10646.0
1,2015-01-01 01:00:00+01:00,449.0,328.0,5196.0,4755.0,158.0,920.0,1009.0,1658.0,7096.0,...,0.971,63,1,309,0,800,clear,sky is clear,16694.0,10929.0
2,2015-01-01 02:00:00+01:00,448.0,323.0,4857.0,4581.0,157.0,1164.0,973.0,1371.0,7099.0,...,0.971,64,1,273,0,800,clear,sky is clear,16191.0,10409.0
3,2015-01-01 03:00:00+01:00,438.0,254.0,4314.0,4131.0,160.0,1503.0,949.0,779.0,7098.0,...,0.971,64,1,273,0,800,clear,sky is clear,15692.0,9340.0
4,2015-01-01 04:00:00+01:00,428.0,187.0,4130.0,3840.0,156.0,1826.0,953.0,720.0,7097.0,...,0.971,64,1,273,0,800,clear,sky is clear,15647.0,8784.0


In [33]:
df.columns

Index(['time', 'generation_biomass', 'generation_fossil_brown_coal/lignite',
       'generation_fossil_gas', 'generation_fossil_hard_coal',
       'generation_fossil_oil', 'generation_hydro_pumped_storage_consumption',
       'generation_hydro_run-of-river_and_poundage',
       'generation_hydro_water_reservoir', 'generation_nuclear',
       'generation_other', 'generation_other_renewable', 'generation_solar',
       'generation_waste', 'generation_wind_onshore', 'total_load_actual',
       'price_day_ahead', 'price_actual', 'city_name', 'temp', 'pressure',
       'humidity', 'wind_speed', 'wind_deg', 'clouds_all', 'weather_id',
       'weather_main', 'weather_description', 'clean_energy_total',
       'fossil_energies_total'],
      dtype='object')

Need to X-y-split AND train-test-split BEFORE I apply transformations, 
then train transformation on training set only

In [34]:
df = df.sample(frac=1)

In [35]:
y = df['price_actual']
X = df.drop(['price_actual','time', 'city_name'], axis=1)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#### Dividing into numericals and categoricals

In [36]:
X_train_num = X_train.select_dtypes(np.number)
X_train_cat = X_train.select_dtypes(object)

X_test_num = X_test.select_dtypes(np.number)
X_test_cat = X_test.select_dtypes(object)

In [37]:
X_train_num.head()

,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run-of-river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,...,price_day_ahead,temp,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,clean_energy_total,fossil_energies_total
14919,352.0,579.0,4444.0,4898.0,282.0,0.0,569.0,1697.0,7001.0,60.0,...,45.68,19.630,1.018,45,1,360,0,800,15261.0,10615.0
22778,349.0,642.0,9188.0,3463.0,359.0,63.0,677.0,1770.0,6889.0,16.0,...,55.47,30.580,1.014,19,6,220,0,800,15575.0,14017.0
19217,274.0,0.0,3954.0,2002.0,311.0,0.0,1232.0,3446.0,7091.0,63.0,...,32.89,3.000,1.019,74,2,320,0,800,26424.0,6604.0
3730,526.0,979.0,4178.0,5409.0,256.0,107.0,730.0,2629.0,4561.0,84.0,...,50.07,31.696,0.959,24,3,113,0,800,17531.0,11432.0
15156,358.0,217.0,3784.0,3333.0,235.0,959.0,464.0,652.0,7002.0,55.0,...,33.60,18.600,1.015,37,0,46,0,800,12030.0,7982.0


#### Normalizing the numerical data

In [38]:
from sklearn.preprocessing import MinMaxScaler
def EscaladorMinMax(data, MinMaxtransformer):
    X_normalized = MinMaxtransformer.transform(data)
    print(X_normalized.shape)
    X_normalized = pd.DataFrame(X_normalized,columns=data.columns)
    return X_normalized

In [39]:
MinMaxtransformer = MinMaxScaler().fit(X_train_num) # Only run once

In [40]:
X_train_num_escalado = EscaladorMinMax(X_train_num, MinMaxtransformer)
X_train_num_escalado.head()

(28014, 25)


,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run-of-river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,...,price_day_ahead,temp,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,clean_energy_total,fossil_energies_total
0,0.594595,0.579580,0.221823,0.585955,0.628062,0.000000,0.2845,0.174445,0.983701,0.566038,...,0.436506,0.582300,0.791304,0.45,0.058824,1.000000,0.0,0.993377,0.455607,0.289263
1,0.589527,0.642643,0.458620,0.414284,0.799555,0.013929,0.3385,0.181949,0.967964,0.150943,...,0.534474,0.804870,0.756522,0.19,0.352941,0.611111,0.0,0.993377,0.464981,0.424494
2,0.462838,0.000000,0.197364,0.239502,0.692650,0.000000,0.6160,0.354235,0.996347,0.594340,...,0.308516,0.244278,0.800000,0.74,0.117647,0.888889,0.0,0.993377,0.788870,0.129825
3,0.888514,0.979980,0.208545,0.647087,0.570156,0.023657,0.3650,0.270251,0.640860,0.792453,...,0.480436,0.827554,0.278261,0.24,0.176471,0.313889,0.0,0.993377,0.523376,0.321739
4,0.604730,0.217217,0.188879,0.398732,0.523385,0.212027,0.2320,0.067023,0.983842,0.518868,...,0.315621,0.561364,0.765217,0.37,0.000000,0.127778,0.0,0.993377,0.359147,0.184601


In [41]:
X_test_num_escalado = EscaladorMinMax(X_test_num, MinMaxtransformer)
X_test_num_escalado.head()

(7004, 25)


,generation_biomass,generation_fossil_brown_coal/lignite,generation_fossil_gas,generation_fossil_hard_coal,generation_fossil_oil,generation_hydro_pumped_storage_consumption,generation_hydro_run-of-river_and_poundage,generation_hydro_water_reservoir,generation_nuclear,generation_other,...,price_day_ahead,temp,pressure,humidity,wind_speed,wind_deg,clouds_all,weather_id,clean_energy_total,fossil_energies_total
0,0.815878,0.312312,0.237546,0.688838,0.710468,0.336723,0.6375,0.278988,0.998735,0.839623,...,0.404183,0.730680,0.773913,0.21,0.176471,0.500000,0.0,0.993377,0.507673,0.333188
1,0.550676,0.000000,0.285165,0.426486,0.538976,0.000000,0.3910,0.187500,0.991569,0.537736,...,0.639047,0.772348,0.826087,0.27,0.117647,0.194444,0.2,0.995033,0.441366,0.260961
2,0.770270,0.296296,0.281871,0.247637,0.726058,0.000000,0.8405,0.824322,0.999016,0.830189,...,0.409687,0.661165,0.773913,0.56,0.058824,0.972222,0.0,0.993377,0.780840,0.220416
3,0.826014,0.591592,0.345063,0.933365,0.839644,0.000000,0.3895,0.259046,0.837572,0.811321,...,0.599219,0.867678,0.817391,0.15,0.176471,0.638889,0.0,0.993377,0.478027,0.513575
4,0.910473,0.288288,0.180593,0.340352,0.590200,0.184612,0.2885,0.132812,0.938738,0.839623,...,0.384769,0.775601,0.730435,0.33,0.411765,0.666667,0.0,0.993377,0.515882,0.171165


#### Onehot encoding for the categorical variables

In [42]:
X_train_cat.head()

,weather_main,weather_description
14919,clear,sky is clear
22778,clear,sky is clear
19217,clear,sky is clear
3730,clear,sky is clear
15156,clear,sky is clear


In [43]:
# X_train_cat['weather_description'].unique()

ValueError: Found unknown categories ['thunderstorm with heavy rain'] in column 1 during transform

In [44]:
from sklearn.preprocessing import OneHotEncoder
def OneHotEncoding(data, OneHotEncoder):
    # print(OneHotEncoder.categories_)
    encoded = OneHotEncoder.transform(data).toarray()
    # print(encoded)
    onehot_encoded = pd.DataFrame(encoded)
    return onehot_encoded

In [45]:
encoder = OneHotEncoder(drop='first', handle_unknown='ignore').fit(X_train_cat) # Only run once
# Ahora habría que llamar a la función

In [46]:
X_train_cat['weather_description'].unique()

array(['sky is clear', 'broken clouds', 'light intensity drizzle',
       'scattered clouds', 'light rain', 'few clouds', 'mist', 'fog',
       'light intensity shower rain', 'overcast clouds', 'shower rain',
       'moderate rain', 'proximity thunderstorm', 'thunderstorm',
       'proximity shower rain', 'light snow', 'drizzle', 'snow',
       'light rain and snow', 'heavy intensity rain', 'rain and snow',
       'thunderstorm with rain', 'haze', 'heavy intensity shower rain',
       'thunderstorm with light rain', 'thunderstorm with heavy rain',
       'sleet', 'light intensity drizzle rain', 'rain and drizzle',
       'heavy snow'], dtype=object)

In [47]:
X_test_cat['weather_description'].unique()

array(['sky is clear', 'few clouds', 'broken clouds', 'mist',
       'scattered clouds', 'fog', 'overcast clouds',
       'thunderstorm with light rain', 'moderate rain', 'drizzle',
       'light rain', 'proximity thunderstorm', 'light intensity drizzle',
       'proximity shower rain', 'snow', 'rain and snow', 'sleet',
       'thunderstorm with rain', 'light thunderstorm', 'thunderstorm',
       'light intensity shower rain', 'light snow',
       'heavy intensity rain', 'shower rain', 'haze',
       'heavy intensity drizzle', 'heavy intensity shower rain',
       'light rain and snow'], dtype=object)

In [48]:
X_train_cat_encoded = OneHotEncoding(X_train_cat, encoder)
X_test_cat_encoded = OneHotEncoding(X_test_cat, encoder)
X_train_cat_encoded.head()

c:\Users\JT\anaconda3\lib\site-packages\sklearn\preprocessing\_encoders.py:188: UserWarning: Found unknown categories in columns [1] during transform. These unknown categories will be encoded as all zeros
  warnings.warn(


,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [49]:
X_test_cat_encoded.head()

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0


In [50]:
X_train = pd.concat([X_train_num_escalado, X_train_cat_encoded], axis=1).reset_index(drop=True)
X_test = pd.concat([X_test_num_escalado, X_test_cat_encoded], axis=1).reset_index(drop=True)
y_train.reset_index(drop=True,inplace=True)
y_test.reset_index(drop=True, inplace=True)

In [51]:
X_train.isna().sum()

generation_biomass                      0
generation_fossil_brown_coal/lignite    0
generation_fossil_gas                   0
generation_fossil_hard_coal             0
generation_fossil_oil                   0
                                       ..
32                                      0
33                                      0
34                                      0
35                                      0
36                                      0
Length: 62, dtype: int64

#### Building a Linear Regression Model

In [52]:
from sklearn import linear_model
linear_model = linear_model.LinearRegression()

In [53]:
X_train.shape, y.shape

((28014, 62), (35018,))

In [54]:
linear_model.fit(X_train,y_train)

c:\Users\JT\anaconda3\lib\site-packages\sklearn\utils\validation.py:1854: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


LinearRegression()

In [55]:
from sklearn.metrics import r2_score
predictions = linear_model.predict(X_train)
r2_score(y_train, predictions)

c:\Users\JT\anaconda3\lib\site-packages\sklearn\utils\validation.py:1854: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.5976687592854755

In [56]:
predictions_test = linear_model.predict(X_test)
r2_score(y_test, predictions_test)

c:\Users\JT\anaconda3\lib\site-packages\sklearn\utils\validation.py:1854: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


0.5713099052566545

In [57]:
y_test[:5]

0    63.38
1    70.74
2    39.95
3    74.66
4    49.02
Name: price_actual, dtype: float64

In [58]:
predictions_test[:5]

array([54.49902344, 68.65332031, 61.8828125 , 69.42773438, 49.61230469])